In [5]:
#!/usr/bin/env python
import os
import torch
import numpy as np

from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# Definicija iste arhitekture (morate koristiti istu definiciju kao u treningu)
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU

class BasicBlockQ(torch.nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, bit_width=4):
        super(BasicBlockQ, self).__init__()
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=3, stride=stride,
                                 padding=1, bias=False, weight_bit_width=bit_width)
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.relu1 = QuantReLU(bit_width=bit_width)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=1,
                                 padding=1, bias=False, weight_bit_width=bit_width)
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.relu2 = QuantReLU(bit_width=bit_width)
        self.shortcut = torch.nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = torch.nn.Sequential(
                QuantConv2d(in_planes, planes, kernel_size=1, stride=stride,
                            bias=False, weight_bit_width=bit_width),
                torch.nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = self.relu1(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu2(out)
        return out

class QuantResNet18(torch.nn.Module):
    def __init__(self, in_channels=3, bit_width=4, num_classes=10):
        super(QuantResNet18, self).__init__()
        self.in_planes = 16
        self.conv1 = QuantConv2d(in_channels, 16, kernel_size=3, stride=1,
                                 padding=1, bias=False, weight_bit_width=bit_width)
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.relu = QuantReLU(bit_width=bit_width)
        
        self.layer1 = self._make_layer(16, blocks=2, stride=1, bit_width=bit_width)
        self.layer2 = self._make_layer(32, blocks=2, stride=2, bit_width=bit_width)
        self.layer3 = self._make_layer(64, blocks=2, stride=2, bit_width=bit_width)
        
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1,1))
        self.fc = QuantLinear(64, num_classes, bias=True, weight_bit_width=bit_width)
    
    def _make_layer(self, planes, blocks, stride, bit_width):
        layers = []
        layers.append(BasicBlockQ(self.in_planes, planes, stride=stride, bit_width=bit_width))
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockQ(self.in_planes, planes, stride=1, bit_width=bit_width))
        return torch.nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def asymmetric_quantize(arr, num_bits=8):
    qmin = 0
    qmax = 2**num_bits - 1
    beta = np.min(arr)
    alpha = np.max(arr)
    scale = (alpha - beta) / qmax if (alpha-beta)!=0 else 1.0
    zero_point = np.clip(-beta/scale, 0, qmax).round().astype(np.int8)
    quantized_arr = np.clip(np.round(arr / scale + zero_point), qmin, qmax).astype(np.float32)
    return quantized_arr

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    
    # Inicijaliziramo i učitavamo model
    model = QuantResNet18(in_channels=3, bit_width=4, num_classes=10)
    model.load_state_dict(torch.load("model_resnet18.pth"))
    model = model.to(device)
    model.eval()
    print("Model loaded from 'model_resnet18.pth'.")
    
    # Postavke za export
    root_dir = "./export"
    os.makedirs(root_dir, exist_ok=True)
    onnx_filename = os.path.join(root_dir, "resnet_part1.onnx")
    onnx_clean_filename = os.path.join(root_dir, "resnet_part1_clean.onnx")
    finn_filename = os.path.join(root_dir, "resnet_ready_finn.onnx")
    
    # Kreiramo dummy ulaz – za CIFAR-10 (3 kanala, 32x32)
    dummy_np = np.random.rand(1, 3, 32, 32).astype(np.float32)
    dummy_np = asymmetric_quantize(dummy_np, num_bits=8)
    print("Max value in quantized input:", np.max(dummy_np))
    scale = 1.0
    input_t = torch.from_numpy(dummy_np * scale)
    
    print("Exporting model to QONNX format...")
    try:
        export_qonnx(model, export_path=onnx_filename, input_t=input_t)
        print(f"Model exported to ONNX file: {onnx_filename}")
    except Exception as e:
        print("Error during ONNX export:", e)
    
    print("Cleaning ONNX model with qonnx_cleanup...")
    try:
        qonnx_cleanup(onnx_filename, out_file=onnx_clean_filename)
        print(f"Cleaned ONNX model saved to: {onnx_clean_filename}")
    except Exception as e:
        print("Error during ONNX model cleanup:", e)
    
    print("Converting QONNX to FINN model...")
    try:
        model_wrapper = ModelWrapper(onnx_clean_filename)
        model_wrapper = model_wrapper.transform(ConvertQONNXtoFINN())
        model_wrapper.save(finn_filename)
        print("FINN model saved to:", finn_filename)
    except Exception as e:
        print("Error during FINN conversion:", e)

if __name__ == '__main__':
    main()


Using device: cpu


FileNotFoundError: [Errno 2] No such file or directory: 'model_resnet18.pth'